Beat-n game with 10 pt limit

## Spud code

In [2]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JLD2

In [3]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [4]:
const MXV = 9
const MNV = 1

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [8]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [9]:
lib0 = get_library(10)
lib0

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [10]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [11]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [12]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end
nlib = length(lib)

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

74

In [13]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [14]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [15]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));

In [16]:
libi = [spud2int(ff) for ff in lib];

In [17]:
int2ind[313111]

73

In [18]:
as = [1,2,3]
as[end]

3

In [19]:
function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]==1
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

eval_beatn (generic function with 1 method)

In [20]:
as = [int2ind[ff] for ff in [214111, 112222, 313111, 113311]]
bs = [int2ind[ff] for ff in [121411, 121321, 113212, 213121]]

4-element Vector{Int64}:
 41
 40
 27
 65

In [21]:
eval_beatn(payoffs, as[1:3], bs[1:2])

2

In [22]:
eval_beatn(payoffs, bs[1:3], as[1:3])

2

In [23]:
eval_beatn(payoffs, as[1:4], bs[1:3])

2

In [24]:
eval_beatn(payoffs, bs[1:4], as[1:4])

3

In [25]:
size(payoffs)[1]

74

In [26]:
# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

try_beatn (generic function with 1 method)

In [27]:
for i in try_beatn(payoffs, as, bs)
    print(i)
end

00000000000000000000000011120110021211220011100001110000000000000120000000

In [28]:
# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

lose_or_not (generic function with 1 method)

In [29]:
lose_or_not(payoffs, as[1:2], bs[1:2])

0

In [30]:
lose_or_not(payoffs, as[1:3], bs[1:3])

0

In [31]:
lose_or_not(payoffs, as[1:4], bs[1:4])

1

In [32]:
function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

legal_moves (generic function with 1 method)

In [33]:
# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

choose_random (generic function with 1 method)

In [34]:
# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

choose_aggro (generic function with 1 method)

In [35]:
function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end

display_game (generic function with 1 method)

In [36]:
a_win, as, bs = play_random_vs_aggro(payoffs, [int2ind[111133]], [int2ind[222211]])
display_game(payoffs, as, bs)

111133 {0}. 222211 {1}. 
131311 {1}. 111223 {2}. 
213112 {2}. 213121 {3}. 
121321 {3}. 113311 {3}. 
214111 {3}. 


In [37]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# for a in 1:nlib
#     for b in 1:nlib
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a], [b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [38]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)
#     js = findall(wins[i, :].==minimum(wins[i, :]))
#     print(libi[i])
#     print(" ")
#     print(libi[js])
#     print(" ")
#     println(minimum(wins[i, :])/n_games)
# end

In [39]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a_moves = legal_moves(payoffs, [a1], [b1])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1], [a1, a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a], [b1,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [40]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [41]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2],[b1,b2])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
312121 {0}. 131212 {2}. 
131311 {3}. 113122 {3}. 
131221 {4}. 


In [42]:
# n_games = 1000
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a_moves = legal_moves(payoffs, [a1,a2], [b1,b2])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2], [a1,a2,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a], [b1,b2,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [43]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [44]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3],[b1,b2,b3])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
111133 {1}. 212311 {2}. 
312121 {2}. 313111 {3}. 
113212 {4}. 212122 {4}. 


In [45]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a3 = int2ind[121321]
# b3 = int2ind[121222]
# a_moves = legal_moves(payoffs, [a1,a2,a3], [b1,b2,b3])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2,b3], [a1,a2,a3,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a], [b1,b2,b3,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [46]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [47]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111133 {2}. 
112231 {2}. 113221 {2}. 
212131 {2}. 214111 {3}. 
121132 {3}. 121312 {4}. 


## MCTS

In [48]:
function intarr_to_string(as::Array{Int64})::String
    return join([convert(Char, a+32) for a in as])
end

function string_to_intarr(s::String)::Array{Int64}
    return [convert(Int64, s[i])-32 for i in 1:length(s)]
end

# s=intarr_to_string([i for i in 1:75])
# as=string_to_intarr(s)
# for i in as
#     print(i)
#     print(" ")
# end

function split_arrs(zs::Array{Int64})::Tuple{Array{Int64},Array{Int64}}
    if mod(length(zs), 2)==0
        return (zs[1:2:(end-1)], zs[2:2:end])
    else
        return (zs[1:2:end], zs[2:2:(end-1)])
    end
end

function merge_arrs(as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    zs = Array{Int64}(undef, length(as)+length(bs))
    if mod(length(zs), 2)==0
        zs[1:2:(end-1)]=as
        zs[2:2:end]=bs
    else
        zs[1:2:end] = as
    zs[2:2:(end-1)] = bs
    end
    return zs
end

merge_arrs (generic function with 1 method)

In [262]:
function propagate_win(tree::Dict{String, Vector{Int64}}, leaf::String, win::Int64)::Dict{String, Vector{Int64}}
    for i in 0:length(leaf)
        if i==0
            node=""
        else
            node=leaf[1:i]
        end
        tree[node][2] += 1
        tree[node][1] += win
    end
    return tree
end

function append_move_to_leaf(leaf::String, move::Int64)::String
    zs = string_to_intarr(leaf)
    append!(zs, move)
    return intarr_to_string(zs)
end

function get_children(payoffs::Array{Int64}, leaf::String)::Array{String}
    # get arrays
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate successors
    if player==1
        cands = legal_moves(payoffs, as, bs)
    else
        cands = legal_moves(payoffs, bs, as)
    end
    return [append_move_to_leaf(leaf, c) for c in cands]
end

get_children (generic function with 1 method)

In [263]:
function check_and_propagate_terminal(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    children = get_children(payoffs, leaf)
    if length(children) == 0
        # indicate terminal
        tree[leaf][3]=1
        if mod(length(leaf), 2)==0
            #p1 lost
            tree = propagate_win(tree, leaf, 0)
        else
            #p1 won
            tree = propagate_win(tree, leaf, 1)
        end
    end
    return tree
end

check_and_propagate_terminal (generic function with 1 method)

In [264]:
function try_add_node(tree::Dict{String, Vector{Int64}}, node::String)::Dict{String, Vector{Int64}}
    if haskey(tree, node)
        return tree
    else
        tree[node] = [0,0,0]
        return tree
    end
end

function query_node(tree, node)::Array{Int64}
    if haskey(tree, node)
        return tree[node]
    else
        return [0,0,0]
    end
end

query_node (generic function with 1 method)

In [265]:
function expand_leaf(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    # check if leaf is terminal
    tree = check_and_propagate_terminal(payoffs, tree, leaf)
    if tree[leaf][3]==1
        return tree
    end
    # get arrays for leaf
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate random successor
    if player==1
        cands = legal_moves(payoffs, as, bs)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    else
        cands = legal_moves(payoffs, bs, as)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    end
    try_add_node(tree, newleaf)
    # play the match
    as, bs = split_arrs(string_to_intarr(newleaf))
    a_win, as2, bs2 = play_random_vs_aggro(payoffs, as, bs)
    # propagate result
    if a_win == 1
        tree = propagate_win(tree, newleaf, 1)
    else
        tree = propagate_win(tree, newleaf, 0)
    end
    return tree
end


expand_leaf (generic function with 2 methods)

In [266]:
function sample_next_node(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, 
        explore_const::Float64=1.4)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if minimum(counts) == 0
        sel = children[rand(findall(counts .== 0))]
        return sel
    end
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i] + explore_const * sqrt((log(n)/counts[i]))
        end
    end
    sel = children[rand(findall(scores .== maximum(scores)))]
    return sel
end

sample_next_node (generic function with 2 methods)

In [267]:
function random_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String="")::String
    while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
        node = sample_next_node(payoffs, tree, node)
    end
    return node
end

random_leaf (generic function with 3 methods)

In [268]:
function display_node(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    print("Node: ")
    println(node)
    as, bs = split_arrs(string_to_intarr(node))
    display_game(payoffs, as, bs)
    print("Stats: ")
    print(query_node(tree,node)[1])
    print("/")
    print(query_node(tree,node)[2])
    print("=")
    s = string(query_node(tree,node)[1]/query_node(tree,node)[2])
    println(s[1:min(5, length(s))])
    if query_node(tree,node)[3]==1
        println("[terminal] ")
    end
end

function display_path(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    for i in 0:length(leaf)
        if i==0
            display_node(payoffs, tree, "")
        else
            display_node(payoffs, tree, leaf[1:i])
        end
        println()
    end
end

function display_choices(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    children = get_children(payoffs, node)
    display_node(payoffs, tree, node)
    for c in children
        println()
        display_node(payoffs, tree, c)
    end
end

display_choices (generic function with 1 method)

In [269]:
#tree = load_object("beatn10_tree.jld2")

tree = Dict("" => [0,0,0])


Dict{String, Vector{Int64}} with 1 entry:
  "" => [0, 0, 0]

In [270]:
tree = expand_leaf(payoffs, tree, "")

Dict{String, Vector{Int64}} with 2 entries:
  "@" => [0, 1, 0]
  ""  => [0, 1, 0]

In [271]:
node = random_leaf(payoffs, tree, "")

""

In [ ]:
for metaiter in 1:100
    s = ""
    for iter in 1:10000
        s = string(s,".")
        node = random_leaf(payoffs, tree, "")
        try_add_node(tree, node)
        s = string(s, node, ".")
        tree = expand_leaf(payoffs, tree, node)
    end
    println(s[end-100:end])
    save_object("beatn10_tree.jld2", tree)
end

HP$8..HYNV..H\5N..H9`Q..HT#%..HT>I..HRSY..HLV_..HL3G..HLFB..HL5a..HLZb..HLPC..HL2&..HLdj..HL>L..HLiW.
..ii0^..ii0)..ii-j..iiLF..iiL9..iiD4..iiD>..iiUf..iiIN..ii&V..ii"X..ii4K..ii=A..iiCO..ii6U..4C(..Z)7.
i/55..i/Qf..i/bR..i[Ha..i[0F..i[&4..i[?\..i[Z=..i[Cf..i[XJ..i['2..i[4`..i[(4..i[$b..i[+7..i[:C..i[FO.
.iiH&..iiH8..iHccX..iHcb\..iHcaA..iHc4E..iHc8=..iH04..iH7i..iHW&..iHQ'..iHZK..iH=A..iH5J..iH]9..iHhQ.
ci]..TcaY..TKa..TKT..TKY..TKJ..TK!..T1"Q..TBD,..TB[E..TH-#..TH-)..TH,1..TcaG..TcJH..Tc8W..Th2M..Th>+.
CiP..H6bJ..HHGh..HO[:..HOHZ..HDb?..Hc`EG..HS>R..H"D(..HjaR..Hjg0..HGaB..H[cN..H[i@..H[BR..H[T4..H[8=.
cP%c..cP/?..cPB(..cP_J..c[.A..c[*0..c@^a..c@N!..cjeI..cj^9..cjf:..c)QC..cj:C..cj2@..cjZV..c5<I..c5#%.
c&]7..c&J4..c&M=..c&_;..c&!G..c&NO..c&#L..c&Kb..c&a;..c&^;..c&g^..c&9=..c&YC..c&/2..c&<U..c&dE..c&I,.
cc+G..cc$h..cc1C..cc,H..cc7V..ccTF..ccTb..cc:h..ccdK..ccNC..ccVU..cci2..ccf9..ccid..cc`T..cc[[..cciB.
"">..ciRG..ci)8>..ciHS7..ciHAX..ciIFB..ciHAa..ciH::..ci*UZ..ci+i..ci+Q..ci+d..ci+f

In [377]:
node = random_leaf(payoffs, tree, "")

"G<."

In [378]:
node = random_leaf(payoffs, tree, "")
display_path(payoffs, tree, node)

Node: 
Stats: 121315/295001=0.411

Node: G
121312 {0}. 
Stats: 1394/3480=0.400

Node: G<
121312 {0}. 113221 {0}. 
Stats: 14/24=0.583

Node: G<1
121312 {0}. 113221 {0}. 
112141 {0}. 
Stats: 0/0=NaN



In [379]:
length(tree)

538893

In [62]:
# node = ""
# while tree[node][2] > 1 && tree[node][3] != 1
#     node = sample_next_node(payoffs, tree, node)
#     display_leaf(payoffs, tree, node)
#     println()
# end


In [63]:
#display_choices(payoffs, tree, "HG`i\"")